# Introducing FEniCS PDE Support in CUQI 

Here we build a Bayesian problem to infer the conductivity (this is a simplification of one of the steps needed in solving Acousto-electric tomography [<sup>1</sup>](#fn1).).


<span id="fn1"><font size="1.5"> 1. Li, C., Karamehmedović, M., Sherina, E., & Knudsen, K. (2021). Levenberg–Marquardt Algorithm for Acousto-Electric Tomography based on the Complete Electrode Model. Journal of Mathematical Imaging and Vision, 63(4), 492-502.</font></span>
## Learning objectives of this notebook:
- Load component of a FEniCS based testproblem
- Build and solve the Bayesian problem
- Explore the effect of changing observation operator

## The forward model

The PDE model we consider here is a 1D steady-state diffusion problem:

$$ \frac{d}{dx} \left(\theta(x) \frac{d}{dx} u(x)\right) = 0 \in [0,L],$$

$$u(0)=0, u(L)=8$$


where $\theta(x)$ is the conductivity, $u(x)$ is the PDE solution (potential) and $L$ is the domain length.

## The Bayesian parameters and the data

The goal is to infer the conductivity profile $\theta$ given observed data $d$ everywhere of the domain. These observation can be of the potential directly, i.e. $d=u$, or a function of the potential (e.g. power density,  $d=\theta|\nabla u|^2$ ). 

The data $d$ is then given by:

$$ d = \mathcal{G}(\theta) + \eta$$


where $\eta$ is the measurement noise and $\mathcal{G}$ is the forward model operator which maps the source term to the observations. We model the prior on $\theta$ as a Gaussian Markov Random Field and the noise as a Gaussian noise.

## The discretization 

We use finite element discretization of the model above where the solution and the parameters are approximated in a first order Lagrange polynomial space.

## The code

To solve this problem using `cuqi.fenics` module, we need to perform the following steps:
- Load the test problem components
- Define the posterior and solve the inverse problem:
    - Define the likelihood
    - Define the prior
    - Define the posterior
    - Sample the posterior

We start by importing the libraries we need

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append("../../")
import cuqi

###  Load the test problem components

We set the problem parameters.

In [ ]:
mapping = 'exponential' # Parametrize the conductivity to ensure positivity
dim = 30                 # Problem dimension
L = 1                    # Domain length 

myExactSolution = 'smooth_step' # True solution choice
observation_operator = None      # When observation operator is None, we observe the solution u directly
SNR = 1000                     # signal to noise ratio

We load the components of the test problem `cuqi.fenics.testproblem.FEniCSDiffusion1D`.

In [ ]:
model, data, problemInfo = \
    cuqi.fenics.testproblem.FEniCSDiffusion1D.get_components(dim=dim,\
                                                             exactSolution=myExactSolution,\
                                                             observation_operator=observation_operator,\
                                                             SNR=SNR,\
                                                             mapping=mapping,\
                                                             left_bc=1,\
                                                             right_bc=8,\
                                                             endpoint=L)

Lets take a look at what we obtain from the test problem. We view the `model`:

In [ ]:
model

and the `model.pde`:

In [ ]:
model.pde

and explore a little bit more by looking at `model.pde` parameters and methods

In [ ]:
vars(model.pde).keys() 

In [ ]:
help(model.pde)

Lets look at the `problemInfo`:

In [ ]:
problemInfo

We now plot the exact data, noisy data and the exact solution: 

In [ ]:
model.range_geometry.plot(data,is_par=True, label='noisy data');
model.range_geometry.plot(problemInfo.exactData,is_par=True, label='exact data');
plt.legend();
plt.figure();
model.domain_geometry.plot(problemInfo.exactSolution,is_par=True, label='exact solution');
plt.legend();

### Define the posterior and solve the inverse problem:

#### Define the likelihood

In [ ]:
sigma = np.linalg.norm(problemInfo.exactData)/SNR 
likelihood = cuqi.distribution.GaussianCov(model, sigma**2*np.eye(model.range_dim)).to_likelihood(data)

#### Define the prior


In [ ]:
prior = cuqi.distribution.GMRF(np.zeros(model.domain_dim),25,1,'zero',\
                               geometry=model.domain_geometry)

#### Define the posterior

We define the posterior using the prior and the likelihood:

In [ ]:
posterior = cuqi.distribution.Posterior(likelihood, prior)

#### Sample the posterior

We finally sample the posterior:

In [ ]:
Ns = 5000
np.random.seed(0)
sampler = cuqi.sampler.pCN(posterior)
samples = sampler.sample_adapt(Ns)

And plot:

In [ ]:
samples.plot_ci(95, plot_par=True, exact=problemInfo.exactSolution)
plt.xticks(np.arange(prior.dim)[::5],['v'+str(i) for i in range(prior.dim)][::5]);

Note that what we show in the plot above is the FEM coefficient $a_i$ where:

<font size="4">$$\theta = e^{\sum_i^\texttt{prior.dim}a_i \phi_i}$$</font>

and $\phi_i$ are the basis functions. In this case of approximating on first order Lagrange polynomial space, the coefficients are equal to the function values at the nodes.

Let's look at the chains:

In [ ]:
samples.plot_chain([1, 4, 15, 20, 30]);

We can apply `burnthin` to remove the burn-in and plot the credibility interval again:

In [ ]:
new_samples = samples.burnthin(500)
new_samples.plot_ci(95,  plot_par=True, exact=problemInfo.exactSolution)
plt.xticks(np.arange(prior.dim)[::5],['v'+str(i) for i in range(prior.dim)][::5]);

### Explore the effect of changing observation operator

* We can try observing the power density ($\theta|\nabla u|^2$) by setting `observation_operator='power_density'`